In [26]:
# Dependencies and set up
import pandas as pd
import numpy as np
import json

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
from sqlalchemy import inspect

# City of Chicago -- Crimes - from 2001 to 2021

In [2]:
# Raw crime table
table = pd.read_csv('Crimes_-_2001_to_Present.csv') 
table.head(2)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.6700,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.7654,"(41.895080471, -87.765400451)"


In [5]:
# table.drop((table['Year'] < 2010).index, inplace=True)

In [3]:
table[(table.Year>2010) & (table.Year<2021)]

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7371605,12441154,JE323342,02/15/2020 07:15:00 PM,016XX W HARRISON ST,0880,THEFT,PURSE-SNATCHING,STREET,False,False,...,27.0,28.0,06,NaN,NaN,2020,08/03/2021 04:47:34 PM,NaN,NaN,NaN
7371739,12441866,JE319748,08/10/2020 11:00:00 AM,124XX S STATE ST,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,RESIDENCE,False,False,...,9.0,53.0,07,NaN,NaN,2020,08/03/2021 04:47:34 PM,NaN,NaN,NaN
7371847,12441105,JE323190,03/01/2019 08:00:00 AM,069XX S PRAIRIE AVE,0266,CRIMINAL SEXUAL ASSAULT,PREDATORY,APARTMENT,False,True,...,6.0,69.0,02,NaN,NaN,2019,08/03/2021 04:47:34 PM,NaN,NaN,NaN
7372182,12441486,JE323541,12/03/2020 12:00:00 AM,097XX S PEORIA ST,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,...,34.0,73.0,06,NaN,NaN,2020,08/03/2021 04:47:34 PM,NaN,NaN,NaN


In [4]:
table.reset_index(inplace=True)

In [5]:
table.head()

,index,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [6]:
# Groupby Primary type
# df = table.groupby(['Primary Type']).count()['Description']
# df

In [7]:
# Dropping table
table = table.drop(columns=['Case Number', 'Location', 'Updated On', 'Block', 'Location Description', 'Community Area', 'Beat', 'District', 'Ward', 'FBI Code', 'X Coordinate', 'Y Coordinate' ])
table.head()

,index,ID,Date,IUCR,Primary Type,Description,Arrest,Domestic,Year,Latitude,Longitude
0,0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,2015,41.815117,-87.670000
1,1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,False,False,2015,41.895080,-87.765400
2,2,11646166,09/01/2018 12:01:00 AM,0810,THEFT,OVER $500,False,True,2018,NaN,NaN
3,3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,2015,41.937406,-87.716650
4,4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,False,True,2015,41.881903,-87.755121


In [8]:
# Dropping NAN from latitude
table = table[table['Latitude'].notna()]
table.head()

,index,ID,Date,IUCR,Primary Type,Description,Arrest,Domestic,Year,Latitude,Longitude
0,0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,2015,41.815117,-87.670000
1,1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,False,False,2015,41.895080,-87.765400
3,3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,2015,41.937406,-87.716650
4,4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,False,True,2015,41.881903,-87.755121
5,5,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,False,False,2015,41.744379,-87.658431


In [10]:
table['Month'] = table['Date'].str[0:2].astype(int)
table = table.drop(['index'], axis=1)
table.head()

,ID,Date,IUCR,Primary Type,Description,Arrest,Domestic,Year,Latitude,Longitude,Month
0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,2015,41.815117,-87.670000,9
1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,False,False,2015,41.895080,-87.765400,9
3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,2015,41.937406,-87.716650,9
4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,False,True,2015,41.881903,-87.755121,9
5,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,False,False,2015,41.744379,-87.658431,9


# Postgres connection - Chicago crime table

In [16]:
# Remember to change to your Postgres password
rds_connection_string = "postgres:123@localhost:5432/Project_03"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [19]:
table.to_sql(name='Chicago_crime', con=engine, if_exists='append', index=False)